# Preprocessing
### Imports

New Python versions sometimes require some extra installations found in the code below

In [ ]:
!pip install standard-sunau standard-aifac

In [ ]:
#Set Dir 
import sys, os
sys.path.append(os.path.abspath('..'))

# Torch
import torch
from torch.utils.data import DataLoader, Dataset
# Utils
import numpy as np
from numpy import ndarray
import logging
# Base Scripts
from Libraries.Utils import *
from Conf import *


### Config

In [10]:
training_data_name: str = "training_2000_1_1"
n_samples: int = 2000
noise_percentage: float = 0.2
overlap: int = 2
logging_level: int = LIGHT_DEBUG
logging.basicConfig(level=logging_level, format='%(asctime)s - %(levelname)s - %(message)s')
logger: logging.Logger = logging.getLogger(__name__)

### Processing

In [12]:
filenames = get_filenames_from_folder(DATA_PATH, "wav")
data: list = []
remaining_samples: int = n_samples
for i in range(len(filenames)):
    file = load_audio_file(os.path.join(DATA_PATH, filenames[i]), SAMPLE_RATE, True)
    file = split_audiofile(file, TIME_FRAME_S, SAMPLE_RATE, overlap)
    file = audio_splits_to_spectograms(file, LEN_FFT, LEN_HOP)
    file = normalize(file, -1, 1)
    data.append(file)
    if file.shape[0] < remaining_samples:
        remaining_samples -= file.shape[0]
    else: break

data: ndarray = np.vstack(data)

data = dimension_for_VAE(data)[:n_samples]
logger.info(f"Processed data of shape: {data.shape}")

2025-03-03 00:55:59,698 - LIGHT_DEBUG - Got filenames ['Am Bach 2024.wav', 'Cutting Mixes mix.wav', 'DA2407_ADO.wav'] from ../Data
2025-03-03 00:56:32,645 - LIGHT_DEBUG - Loaded audio form ../Data\Am Bach 2024.wav of dimensions: (333370680,), sr: 44100
2025-03-03 00:56:34,027 - LIGHT_DEBUG - Split audio to: (1259, 352800)
2025-03-03 00:56:34,246 - LIGHT_DEBUG - Started STFT on splits


2025-03-03 00:57:36,000 - LIGHT_DEBUG - Processed Splits: 1250


2025-03-03 00:57:41,678 - LIGHT_DEBUG - Created spectograms of splits: (1259, 1025, 690)
2025-03-03 00:58:24,740 - LIGHT_DEBUG - Normalized to range: [-1,1]
2025-03-03 00:58:40,025 - LIGHT_DEBUG - Loaded audio form ../Data\Cutting Mixes mix.wav of dimensions: (265496665,), sr: 44100
2025-03-03 00:58:40,550 - LIGHT_DEBUG - Split audio to: (1003, 352800)
2025-03-03 00:58:40,687 - LIGHT_DEBUG - Started STFT on splits


2025-03-03 00:59:19,000 - LIGHT_DEBUG - Processed Splits: 1000


2025-03-03 00:59:23,133 - LIGHT_DEBUG - Created spectograms of splits: (1003, 1025, 690)
2025-03-03 00:59:43,937 - LIGHT_DEBUG - Normalized to range: [-1,1]
2025-03-03 01:00:19,232 - INFO - Processed data of shape: (2000, 1024, 672)


### Saving

In [13]:
save_training_data(data, f"{DATA_PATH}/{training_data_name}.npy")

2025-03-03 01:04:59,380 - LIGHT_DEBUG - Saved ndarray to:../Data/training_2000_1_1.npy
